In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

In [ ]:
dataset = pd.read_csv('../input/avocado-prices/avocado.csv')
dataset.head(10)

In [ ]:
dataset.info()

In [ ]:
nan = dataset.isnull().sum()
nan

In [ ]:
ds_sample = dataset.sample(100)
ds_sample

In [ ]:
# hangi kolonda kaç tane null var?

ds_isnull = dataset.isnull().sum()
ds_isnull


In [ ]:
df_desc = dataset.describe() 

In [ ]:
# korelasyon matrisi

corrmat = dataset.corr()
corrmat

In [ ]:
# korelasyon matrisinin görselleştirilmesi

plt.subplots(figsize=(15,12))
sns.heatmap(corrmat, vmax=1, cmap="Blues", square=True)

In [ ]:
#"AvocadoPrice" ve diğer  özelliklerinin korelasyon matrisinin oluşturulması

averagePrice = dataset["AveragePrice"]
corr_averagePrice = dataset.corrwith(averagePrice, axis=0)
corr_averagePrice = pd.DataFrame(corr_averagePrice)
corr_averagePrice.rename (columns = {0: 'AveragePrice'}, inplace = True)

#Oluşturulan korelasyon matrisinin görselleştirilmesi
plt.subplots(figsize=(15,12))
sns.set(font_scale=1.1)
sns.heatmap(corr_averagePrice, vmax=1, cmap="Blues",annot=True)

**DATA CLEAN**

In [ ]:
df = dataset.drop(columns = ["Unnamed: 0"])
df

In [ ]:
#%% total bags =? small + large + xlarge

df_bags = df[["Total Bags", "Small Bags", "Large Bags", "XLarge Bags"]]
df_bags["total2"] = df_bags["Small Bags"] + df_bags["Large Bags"] + df_bags["XLarge Bags"]
df_bags["diff"] = df_bags["Total Bags"] - df_bags["total2"]
df_bags

In [ ]:
df = df.drop(columns = ["Total Bags"])

In [ ]:
regions = df["region"].unique()
regions

In [ ]:
df["date2"] = pd.to_datetime(df["Date"])
df["month"] = pd.DatetimeIndex(df["date2"]).month
df = df.drop(columns=["Date"])
df = df.drop(columns=["date2"])

In [ ]:
df

In [ ]:
#"AvocadoPrice" ve diğer  özelliklerinin korelasyon matrisinin oluşturulması

averagePrice = df["AveragePrice"]
corr_averagePrice = df.corrwith(averagePrice, axis=0)
corr_averagePrice = pd.DataFrame(corr_averagePrice)
corr_averagePrice.rename (columns = {0: 'AveragePrice'}, inplace = True)

#Oluşturulan korelasyon matrisinin görselleştirilmesi
plt.subplots(figsize=(15,12))
sns.set(font_scale=1.1)
sns.heatmap(corr_averagePrice, vmax=1, cmap="Blues",annot=True)

In [ ]:
df_dummy = pd.get_dummies(df[["type", "region"]])


df = df.drop(columns=["type"])
df = df.drop(columns=["region"])

df = pd.concat([df,df_dummy], axis=1)

**MODEL**

In [ ]:
y = df["AveragePrice"]
X = df.drop(columns = ["AveragePrice"])

X, y = shuffle(X, y, random_state = 7)

In [ ]:
#  70% of the data will be used for train
#  30% will be used for testing

row_size = df.shape[0]  # int 506
offset = int(row_size * 0.7) # int 354.2

# X_train = X[:] overfit olur 
X_train = X[:offset]  # Generating X_train data
X_test = X[offset:]   # Generating X_test data


y_train = y[:offset]  # Generating y_train data
y_test = y[offset:]  # yGenerating y_test data

In [ ]:
model = RandomForestRegressor(n_estimators = 100, max_depth=5)

tuned = model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred

In [ ]:
# score
score_r2 = metrics.r2_score(y_test, y_pred)
score_r2

In [ ]:
rf_Importance = pd.DataFrame({"Importance":tuned.feature_importances_*100},index = X_train.columns)

s = rf_Importance.sort_values(by = "Importance", axis=0, ascending = False)
fig_dims = (50, 50)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x = "Importance", y =  rf_Importance["Importance"].index, ax=ax, data=s);

**RANDOM FOREST FROM SCRATCH WITH LEAST SQUARES METHOD**

In [ ]:
class RandomForest():   
    
    def __init__(self, n_estimators, max_depth =3):
        
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.multiple = None
    

    def create_tree(self, x, y):
        
        idxs = np.random.randint(len(y), size = len(x))
     
        return DecisionTree(x.iloc[idxs], 
                            y.iloc[idxs],
                            depth = 0,
                            max_depth = self.max_depth)

    def fit(self, x, y):
        
        self.trees = []
        
        for i in range(self.n_estimators):  
            print("{}. tree created".format(i+1))
            self.trees.append(self.create_tree(x, y))
            
        print("All tree created")
        self.find_multiple(x,y)
        
        return self.trees
    
    def find_multiple(self,x,y):

        tree_predictions_train = []
        for tree in self.trees:
            tree_predictions_train.append(tree.prediction(x))  

        tree_predictions_df = pd.DataFrame( tree_predictions_train).transpose()

        tree_predictions_df ["y"] = y
        tree_predictions_df = tree_predictions_df.dropna()

        y = tree_predictions_df["y"]
        y_matrix = np.matrix(y).transpose()

        tp = tree_predictions_df.drop("y", axis="columns")
        tp_matrix = np.matrix(tp)
        tp_matrix_t = np.matrix(tp).transpose()

        square_tp_matrix = np.dot(tp_matrix_t,tp_matrix)

        tp_inv = np.linalg.pinv(square_tp_matrix)


        right = np.dot(tp_matrix_t,y_matrix)

        m = np.dot(tp_inv,right)
        arr_m = np.array(m).tolist()
        multiple = [m[0] for m in arr_m]

        self.multiple = multiple 
 
    def predict(self, x):
        tree_predictions = []
        index = 0
        
        for tree in self.trees:
            tree_predictions.append(round(self.multiple[index],2)*tree.prediction(x))
            index += 1
         
        prediction = np.sum(tree_predictions, axis=0)
        return prediction 

class DecisionTree:
    
    def __init__(self, X, y, depth=0, max_depth=3):
        
        self.X = X
        self.y = y
        self.depth = depth
        self.max_depth = max_depth
        self.split(self.X, self.y, self.depth, self.max_depth)
    
        
    def find_gini(self, y_left, y_right):
        
        left = np.sum((y_left - np.mean(y_left)) ** 2)
        right = np.sum((y_right - np.mean(y_right)) ** 2) 
        
        gini = left + right 
        
        return gini
        
        
    def find_best_rule(self, X, y):
        
        best_feature = None
        best_threshold = None
        min_gini = np.inf
        
        for feature in X.columns:
            
            thresholds = X[feature].unique().tolist()
            thresholds.sort()
            # thresholds = thresholds[1:]
            
            for threshold in thresholds:
                y_left_ix = X[feature] < threshold
                
                y_left = y[y_left_ix]
                y_right = y[~y_left_ix]
                
                t_gini = self.find_gini(y_left, y_right)
                
                if t_gini < min_gini:
                    min_gini = t_gini
                    best_threshold = threshold
                    best_feature = feature
        
        return {'feature': best_feature, 'threshold': best_threshold}
            
   
    def split(self, X, y, depth, max_depth):
        
        if depth == max_depth or len(X) < 2:
            return {'prediction': np.mean(y)}
        
        rule = self.find_best_rule(X, y)
       
        left_ix = X[rule['feature']] < rule['threshold']
        
        rule['left'] = self.split(X[left_ix], y[left_ix], depth+1, max_depth)
       
        rule['right'] = self.split(X[~left_ix], y[~left_ix], depth+1, max_depth) 
        
        self.rules = rule
        
        return rule
  
    def predict_row(self, sample, rules):
        prediction = None
        while prediction is None:
            feature, threshold = rules['feature'], rules['threshold']
            if sample[feature] < threshold:
                rules = rules['left']
            else:
                rules = rules['right']
            prediction = rules.get('prediction', None)
        
        return prediction
        
    def prediction(self, X):
        
        preds = X.apply(self.predict_row, axis='columns', rules = self.rules)
        
        return preds

In [ ]:
model_rf = RandomForest(n_estimators=50, max_depth = 5)
model_rf

In [ ]:
tuned_rf = model_rf.fit(X_train, y_train)

In [ ]:
y_pred = model_rf.predict(X_test)
y_pred

In [ ]:
score_r2_rf = metrics.r2_score(y_test, y_pred)
score_r2_rf